# Deepfake Detection Platform - Audio Models Test

This notebook tests all audio detection models and the Acoustic Guardian Singularity Mode.

In [ ]:
# Setup imports
import os
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import torch
import librosa
import librosa.display

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..'))
sys.path.insert(0, project_root)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# Check CUDA availability
print(f"CUDA available: {torch.cuda.is_available()}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## 1. Load Individual Audio Detector Models

We'll load all the audio detector models: Wav2Vec2, XLSR+SLS, XLSR-Mamba, and TCN-Add.

In [ ]:
# Import audio detector models
from detectors.audio_detector.wav2vec_detector import Wav2VecAudioDetector
from detectors.audio_detector.xlsr_detector import XLSRAudioDetector
from detectors.audio_detector.mamba_detector import MambaAudioDetector
from detectors.audio_detector.tcn_detector import TCNAudioDetector

# Initialize the audio detectors
try:
    print("Loading Wav2Vec2 model...")
    wav2vec_detector = Wav2VecAudioDetector(
        model_name="facebook/wav2vec2-large-960h", 
        confidence_threshold=0.5,
        device=device
    )
    
    print("Loading XLSR+SLS model...")
    xlsr_detector = XLSRAudioDetector(
        model_name="facebook/wav2vec2-xlsr-53", 
        confidence_threshold=0.5,
        device=device
    )
    
    print("Loading XLSR-Mamba model...")
    mamba_detector = MambaAudioDetector(
        model_name="facebook/wav2vec2-xls-r-300m", 
        confidence_threshold=0.5,
        device=device
    )
    
    print("Loading TCN-Add model...")
    tcn_detector = TCNAudioDetector(
        model_name="facebook/wav2vec2-base-960h", 
        confidence_threshold=0.5,
        device=device
    )
    
    print("All models loaded successfully!")
    
except Exception as e:
    print(f"Error loading models: {str(e)}")

## 2. Test Individual Models on Sample Audio Files

In [ ]:
# Define test audio files
real_audio_path = os.path.join(project_root, 'tests', 'test_data', 'Real_Audio')
fake_audio_path = os.path.join(project_root, 'tests', 'test_data', 'Fake_Audio')

# List available audio files
real_audio_files = [os.path.join(real_audio_path, f) for f in os.listdir(real_audio_path) if f.endswith(('.wav', '.mp3', '.ogg'))]
fake_audio_files = [os.path.join(fake_audio_path, f) for f in os.listdir(fake_audio_path) if f.endswith(('.wav', '.mp3', '.ogg'))]

if not real_audio_files:
    print("No real audio files found")
else:
    print(f"Found {len(real_audio_files)} real audio files:")
    for f in real_audio_files:
        print(f"  {os.path.basename(f)}")

if not fake_audio_files:
    print("No fake audio files found")
else:
    print(f"Found {len(fake_audio_files)} fake audio files:")
    for f in fake_audio_files:
        print(f"  {os.path.basename(f)}")

# Select test files
real_audio_test = real_audio_files[0] if real_audio_files else None
fake_audio_test = fake_audio_files[0] if fake_audio_files else None

In [ ]:
# Helper function to display audio waveform and spectrogram
def display_audio(audio_path):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        
        plt.figure(figsize=(12, 6))
        plt.subplot(2, 1, 1)
        librosa.display.waveshow(y, sr=sr)
        plt.title('Waveform')
        
        plt.subplot(2, 1, 2)
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram')
        
        plt.tight_layout()
        plt.show()
        
        return y, sr
    except Exception as e:
        print(f"Error displaying audio: {str(e)}")
        return None, None

In [ ]:
# Display real and fake audio if available
if real_audio_test:
    print(f"Real audio sample: {os.path.basename(real_audio_test)}")
    real_y, real_sr = display_audio(real_audio_test)

if fake_audio_test:
    print(f"Fake audio sample: {os.path.basename(fake_audio_test)}")
    fake_y, fake_sr = display_audio(fake_audio_test)

In [ ]:
# Helper function to test models on audio files
def test_audio_model(detector, audio_path, audio_type):
    print(f"Testing {detector.__class__.__name__} on {audio_type} audio...")
    try:
        result = detector.detect(audio_path)
        is_deepfake = result['is_deepfake']
        confidence = result['confidence']
        correct = (is_deepfake and audio_type == 'fake') or (not is_deepfake and audio_type == 'real')
        
        print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
        print(f"  Confidence: {confidence:.4f}")
        print(f"  Correct: {'✓' if correct else '✗'}")
        
        # Display spectrogram if available
        if 'spectrogram' in result:
            plt.figure(figsize=(10, 4))
            plt.imshow(result['spectrogram'], aspect='auto', origin='lower', cmap='viridis')
            plt.colorbar(format='%+2.0f dB')
            plt.title(f"{detector.__class__.__name__} Spectrogram Analysis (Confidence: {confidence:.4f})")
            plt.tight_layout()
            plt.show()
            
        # Display anomaly regions if available
        if 'anomaly_regions' in result and result['anomaly_regions']:
            print("  Detected anomaly regions:")
            for i, region in enumerate(result['anomaly_regions']):
                print(f"    Region {i+1}: {region['start_time']:.2f}s - {region['end_time']:.2f}s (confidence: {region['confidence']:.4f})")
        
        return result, correct
    except Exception as e:
        print(f"  Error: {str(e)}")
        return None, False

In [ ]:
# Test individual models if test files are available
results = {}

if real_audio_test and fake_audio_test:
    # Test Wav2Vec2 model
    results['wav2vec_real'] = test_audio_model(wav2vec_detector, real_audio_test, 'real')
    results['wav2vec_fake'] = test_audio_model(wav2vec_detector, fake_audio_test, 'fake')
    
    # Test XLSR model
    results['xlsr_real'] = test_audio_model(xlsr_detector, real_audio_test, 'real')
    results['xlsr_fake'] = test_audio_model(xlsr_detector, fake_audio_test, 'fake')
    
    # Test Mamba model
    results['mamba_real'] = test_audio_model(mamba_detector, real_audio_test, 'real')
    results['mamba_fake'] = test_audio_model(mamba_detector, fake_audio_test, 'fake')
    
    # Test TCN model
    results['tcn_real'] = test_audio_model(tcn_detector, real_audio_test, 'real')
    results['tcn_fake'] = test_audio_model(tcn_detector, fake_audio_test, 'fake')
else:
    print("Cannot test models without both real and fake audio samples.")

## 3. Test Audio Ensemble Detector

In [ ]:
# Import the audio ensemble detector
from detectors.audio_detector.ensemble import AudioEnsembleDetector

# Create the ensemble detector with all individual detectors
if 'wav2vec_detector' in locals() and 'xlsr_detector' in locals() and 'mamba_detector' in locals() and 'tcn_detector' in locals():
    detectors = [wav2vec_detector, xlsr_detector, mamba_detector, tcn_detector]
    audio_ensemble = AudioEnsembleDetector(
        detectors=detectors,
        weights=None,  # Use equal weights initially
        threshold=0.5,
        enable_singularity=False  # First test without Singularity Mode
    )
    
    print(f"Created Audio Ensemble Detector with {len(detectors)} models")

In [ ]:
# Test ensemble on real and fake audio
if 'audio_ensemble' in locals() and real_audio_test and fake_audio_test:
    # Test on real audio
    print("Testing Audio Ensemble on real audio...")
    ensemble_real_result = audio_ensemble.predict(real_audio_test)
    is_deepfake = ensemble_real_result['is_deepfake']
    confidence = ensemble_real_result['confidence']
    print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if not is_deepfake else '✗'}")
    
    # Show individual model contributions
    if 'individual_results' in ensemble_real_result:
        print("\nIndividual model contributions:")
        for result in ensemble_real_result['individual_results']:
            model_name = result['model']
            confidence = result['confidence']
            weight = result['weight']
            print(f"  {model_name}: Confidence = {confidence:.4f}, Weight = {weight:.2f}")
    
    # Test on fake audio
    print("\nTesting Audio Ensemble on fake audio...")
    ensemble_fake_result = audio_ensemble.predict(fake_audio_test)
    is_deepfake = ensemble_fake_result['is_deepfake']
    confidence = ensemble_fake_result['confidence']
    print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if is_deepfake else '✗'}")
    
    # Show individual model contributions
    if 'individual_results' in ensemble_fake_result:
        print("\nIndividual model contributions:")
        for result in ensemble_fake_result['individual_results']:
            model_name = result['model']
            confidence = result['confidence']
            weight = result['weight']
            print(f"  {model_name}: Confidence = {confidence:.4f}, Weight = {weight:.2f}")

## 4. Test Acoustic Guardian Singularity Mode

In [ ]:
# Enable Singularity Mode in the ensemble detector
if 'detectors' in locals():
    audio_ensemble_with_singularity = AudioEnsembleDetector(
        detectors=detectors,
        weights=None,
        threshold=0.5,
        enable_singularity=True  # Enable Singularity Mode
    )
    
    print("Created Audio Ensemble Detector with Acoustic Guardian Singularity Mode enabled")

In [ ]:
# Test Singularity Mode on real and fake audio
if 'audio_ensemble_with_singularity' in locals() and real_audio_test and fake_audio_test:
    # Test on real audio
    print("Testing Acoustic Guardian on real audio...")
    try:
        singularity_real_result = audio_ensemble_with_singularity.predict(real_audio_test)
        is_deepfake = singularity_real_result['is_deepfake']
        confidence = singularity_real_result['confidence']
        print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
        print(f"  Confidence: {confidence:.4f}")
        print(f"  Correct: {'✓' if not is_deepfake else '✗'}")
        
        # Show Singularity Mode information
        if 'singularity_mode' in singularity_real_result:
            sm_info = singularity_real_result['singularity_mode']
            print("\nSingularity Mode information:")
            for key, value in sm_info.items():
                if key != 'adaptive_weights':
                    print(f"  {key}: {value}")
        
        # Display enhanced spectrogram if available
        if 'spectrogram' in singularity_real_result:
            plt.figure(figsize=(10, 4))
            plt.imshow(singularity_real_result['spectrogram'], aspect='auto', origin='lower', cmap='viridis')
            plt.colorbar(format='%+2.0f dB')
            plt.title(f"Acoustic Guardian Enhanced Spectrogram (Real Audio)")
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"Error testing Acoustic Guardian on real audio: {str(e)}")
    
    # Test on fake audio
    print("\nTesting Acoustic Guardian on fake audio...")
    try:
        singularity_fake_result = audio_ensemble_with_singularity.predict(fake_audio_test)
        is_deepfake = singularity_fake_result['is_deepfake']
        confidence = singularity_fake_result['confidence']
        print(f"  Prediction: {'Deepfake' if is_deepfake else 'Real'}")
        print(f"  Confidence: {confidence:.4f}")
        print(f"  Correct: {'✓' if is_deepfake else '✗'}")
        
        # Show Singularity Mode information
        if 'singularity_mode' in singularity_fake_result:
            sm_info = singularity_fake_result['singularity_mode']
            print("\nSingularity Mode information:")
            for key, value in sm_info.items():
                if key != 'adaptive_weights':
                    print(f"  {key}: {value}")
        
        # Display enhanced spectrogram if available
        if 'spectrogram' in singularity_fake_result:
            plt.figure(figsize=(10, 4))
            plt.imshow(singularity_fake_result['spectrogram'], aspect='auto', origin='lower', cmap='viridis')
            plt.colorbar(format='%+2.0f dB')
            plt.title(f"Acoustic Guardian Enhanced Spectrogram (Fake Audio)")
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"Error testing Acoustic Guardian on fake audio: {str(e)}")

## 5. Performance Comparison

In [ ]:
# Compare performance across models
if all(var in locals() for var in ['ensemble_real_result', 'ensemble_fake_result', 'singularity_real_result', 'singularity_fake_result']):
    # Collect model performances
    models = []
    real_confidences = []
    fake_confidences = []
    
    # Add individual model results
    for model_name in ['wav2vec', 'xlsr', 'mamba', 'tcn']:
        real_key = f"{model_name}_real"
        fake_key = f"{model_name}_fake"
        
        if real_key in results and fake_key in results:
            real_result, _ = results[real_key]
            fake_result, _ = results[fake_key]
            
            if real_result and fake_result:
                models.append(model_name.capitalize())
                real_confidences.append(real_result['confidence'])
                fake_confidences.append(fake_result['confidence'])
    
    # Add ensemble results
    models.append('Ensemble')
    real_confidences.append(ensemble_real_result['confidence'])
    fake_confidences.append(ensemble_fake_result['confidence'])
    
    # Add Singularity Mode results
    models.append('Acoustic Guardian')
    real_confidences.append(singularity_real_result['confidence'])
    fake_confidences.append(singularity_fake_result['confidence'])
    
    # Plot comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Real audio (lower confidence is better)
    ax1.bar(models, real_confidences)
    ax1.set_title('Real Audio Detection Confidence')
    ax1.set_ylabel('Confidence (lower is better for real)')
    ax1.set_ylim(0, 1)
    ax1.axhline(y=0.5, color='r', linestyle='--')
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    
    # Fake audio (higher confidence is better)
    ax2.bar(models, fake_confidences)
    ax2.set_title('Fake Audio Detection Confidence')
    ax2.set_ylabel('Confidence (higher is better for fake)')
    ax2.set_ylim(0, 1)
    ax2.axhline(y=0.5, color='r', linestyle='--')
    plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()

## 6. Conclusion

In [ ]:
# Calculate performance improvements
if all(var in locals() for var in ['ensemble_real_result', 'ensemble_fake_result', 'singularity_real_result', 'singularity_fake_result']):
    # For real audio (lower confidence is better)
    real_ensemble_conf = ensemble_real_result['confidence']
    real_singularity_conf = singularity_real_result['confidence']
    real_improvement = max(0, real_ensemble_conf - real_singularity_conf)
    
    # For fake audio (higher confidence is better)
    fake_ensemble_conf = ensemble_fake_result['confidence']
    fake_singularity_conf = singularity_fake_result['confidence']
    fake_improvement = max(0, fake_singularity_conf - fake_ensemble_conf)
    
    print("Performance Summary:")
    print(f"Acoustic Guardian improvement on real audio: {real_improvement:.4f} reduction in confidence")
    print(f"Acoustic Guardian improvement on fake audio: {fake_improvement:.4f} increase in confidence")
    print(f"Overall improvement: {(real_improvement + fake_improvement) / 2:.4f}")